In [ ]:
import pandas as pd
from dateutil import relativedelta, parser
from pandas.tseries.offsets import BDay 
from datetime import datetime, date, time
import numpy as np
import pickle
import time
import re
import matplotlib.pyplot as plt
import pandas_datareader.data as reader
from sklearn.model_selection import train_test_split

import os
from torch.utils.tensorboard import SummaryWriter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#path_root = f"{os.getcwd()}"
path_root = "/content/drive/MyDrive/Colab Notebooks"
print(path_root)

/content/drive/MyDrive/Colab Notebooks


In [ ]:
df = pd.read_pickle(f"{path_root}/with_all_ft_clean.pkl")
df.head()

,texts,pct_change1,pct_change2,beta1,beta2,mkt_excess1,mkt_excess2,item 1.01,item 1.02,item 1.03,item 1.04,item 2.01,item 2.02,item 2.03,item 2.04,item 2.05,item 2.06,item 2.10,item 3.01,item 3.02,item 3.03,item 4.01,item 4.02,item 4.04,item 5.01,item 5.02,item 5.03,item 5.04,item 5.05,item 5.07,item 5.08,item 7.01,item 7.02,item 8.01,item 9.01,item 9.02,VIX,label1,label2,SHRCD_12.0,SHRCD_14.0,SHRCD_18.0,SHRCD_21.0,SHRCD_31.0,SHRCD_41.0,SHRCD_44.0,SHRCD_48.0,SHRCD_71.0,SHRCD_72.0,SHRCD_73.0,SHRCD_74.0,ind_21,ind_22,ind_23,ind_31,ind_32,ind_33,ind_42,ind_44,ind_45,ind_48,ind_49,ind_51,ind_52,ind_53,ind_54,ind_55,ind_56,ind_61,ind_62,ind_71,ind_72,ind_81,ind_na,EXCHCD_1.0,EXCHCD_2.0,EXCHCD_3.0,EXCHCD_4.0,Mkt_Cap
0,Item 2.02\nResults of Operations and Financial...,0.039772,0.018824,0.1235,0.12100,0.039502,0.017531,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,86355.36
1,Item 2.02 Results of Operations and Financial ...,0.005894,0.019647,0.9098,0.91085,-0.003435,0.005013,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,20.47,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,125102.08
2,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,-0.019268,-0.019268,0.8742,0.87420,-0.013888,-0.013888,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,19.07,neg,neg,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,126812.46
3,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.077505,0.035918,0.8901,0.88485,0.076161,0.027620,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,16.14,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,139273.80
4,ITEM 1.01.\nENTRY INTO A MATERIAL DEFINITIVE A...,0.076013,0.076013,0.9225,0.92250,0.070040,0.070040,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,13.51,pos,pos,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,144649.28


In [ ]:
def label1(row):
    if row["pct_change1"] > row["mkt_excess1"]:
        return "pos"
    else:
        return "neg"

def label2(row):
    if row["pct_change2"] > row["mkt_excess2"]:
        return "pos"
    else:
        return "neg"

In [ ]:
df["label1"] = df.apply(label1, axis = 1)
df["label2"] = df.apply(label2, axis = 1)

In [ ]:
# Split into train and test data
X_train, X_test, y_train, y_test = train_test_split(df.index, df["label2"],
                                                    stratify=df["label2"], 
                                                    test_size=0.2,
                                                    random_state = 20)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,
                                                    stratify=y_train, 
                                                    test_size=0.2,
                                                    random_state = 20)

In [ ]:
# get column names of categorical features
item_list = [i for i in df.columns if not i.find("item")]
ind_list = [i for i in df.columns if not i.find("ind")]
shrcd_list = [i for i in df.columns if not i.find("SHRCD")]
exc_list = [i for i in df.columns if not i.find("EXCHCD")]

In [ ]:
# combine all feature column names
cols = ['VIX', "Mkt_Cap"]
cols.extend(item_list)
cols.extend(ind_list)
cols.extend(shrcd_list)
cols.extend(exc_list)

In [ ]:
# get numerical features
num_train = df.loc[X_train, cols]
num_val = df.loc[X_val, cols]
num_test = df.loc[X_test, cols]

#get texts
text_train = df.loc[X_train, 'texts']
text_val = df.loc[X_val, 'texts']
text_test = df.loc[X_test, 'texts']

N, D = num_train.shape
print("num_train.shape", num_train.shape, "text_train.shape", text_train.shape)
print("num_val.shape ", num_val.shape, "text_val.shape ", text_val.shape)
print("N:", N, "D:", D)

num_train.shape (13888, 70) text_train.shape (13888,)
num_val.shape  (3473, 70) text_val.shape  (3473,)
N: 13888 D: 70


In [ ]:
from sklearn.preprocessing import StandardScaler

x_scaler = StandardScaler()

# standardize volatility index and market cap by mean and standard deviation (standardize train and test sets separately)
num_train["VIX"] = x_scaler.fit_transform(np.array(num_train["VIX"]).reshape(-1,1))
num_val["VIX"] = x_scaler.transform(np.array(num_val["VIX"]).reshape(-1,1))
num_test["VIX"] = x_scaler.transform(np.array(num_test["VIX"]).reshape(-1,1))
num_train["Mkt_Cap"] = x_scaler.fit_transform(np.array(num_train["Mkt_Cap"]).reshape(-1,1))
num_val["Mkt_Cap"] = x_scaler.transform(np.array(num_val["Mkt_Cap"]).reshape(-1,1))
num_test["Mkt_Cap"] = x_scaler.transform(np.array(num_test["Mkt_Cap"]).reshape(-1,1))

In [ ]:
# get a train set data frame of texts and labels only
tt1 = text_train.to_frame().rename({'texts': 'text'}, axis=1)
tt1.loc[:, 'label'] = y_train

In [ ]:
tt1.to_csv('text_train.csv')

In [ ]:
# get a validation set data frame of texts and labels only
tt2 = text_val.to_frame().rename({'texts': 'text'}, axis=1)
tt2.loc[:, 'label'] = y_val

In [ ]:
tt2.to_csv('text_val.csv')

In [ ]:
# get a test set data frame of texts and labels only
tt3 = text_test.to_frame().rename({'texts': 'text'}, axis=1)
tt3.loc[:, 'label'] = y_test

In [ ]:
tt3.to_csv('text_test.csv')

In [ ]:
import torch
import torchtext
from torchtext.legacy.data import Field, LabelField, TabularDataset
from torch.utils.data import DataLoader, TensorDataset
from torchtext.legacy import data as dl
import torch.nn as nn
import torch.optim as optim

In [ ]:
# create train and validation data loader for numerical features (set shuffle to False to keep same indices with texts)
dataloader2 = DataLoader(TensorDataset(torch.from_numpy(num_train.values)), batch_size=64, shuffle=False)
dataloader2_valid = DataLoader(TensorDataset(torch.from_numpy(num_val.values)), batch_size=64, shuffle=False)
dataloader2_test = DataLoader(TensorDataset(torch.from_numpy(num_test.values)), batch_size=64, shuffle=False)

In [ ]:
num_test.values.shape

(4341, 70)

In [ ]:
# define iterators of numerical features
dataloader2_iter = dataloader2._get_iterator()
dataloader2_valid_iter = dataloader2_valid._get_iterator()
dataloader2_test_iter = dataloader2_test._get_iterator()

In [ ]:
# define the fields with pre-processing pipeline
TEXT = Field(tokenize="spacy",lower = True, sequential=True, batch_first=True,include_lengths=True)
LABEL = LabelField(dtype = torch.float,batch_first=True)

In [ ]:
# define columns names (None is the ID column)
fields = [(None, None),('text',TEXT), ('label', LABEL)]

In [ ]:
# load and pre-process texts and labels
train_data=TabularDataset(path = "text_train.csv",format = 'csv',fields = fields,skip_header = True,)
valid_data=TabularDataset(path = "text_val.csv",format = 'csv',fields = fields,skip_header = True,)
test_data=TabularDataset(path = "text_test.csv",format = 'csv',fields = fields,skip_header = True,)

In [ ]:
len(test_data)

4341

In [ ]:
# build vocabulary with training set using 100d pre-trained GloVe embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [02:47, 5.16MB/s]                           
100%|█████████▉| 398028/400000 [00:20<00:00, 19571.39it/s]

In [ ]:
print(len(TEXT.vocab))
print("---------------")
print(len(LABEL.vocab))

19843
---------------
2


In [ ]:
# create iterators of text data
iterator1 = dl.Iterator(train_data, batch_size=64, shuffle=False)
iterator1_valid = dl.Iterator(valid_data, batch_size=64, shuffle=False)
iterator1_test = dl.Iterator(test_data, batch_size=64, shuffle=False)

In [ ]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, aux_dim, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, usebias):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,
                           batch_first=True,
                           bias = usebias)

        #dense layer
        self.fc = nn.Linear(hidden_dim * 2 + aux_dim, output_dim, bias = usebias)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths, features):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(),batch_first=True, enforce_sorted=False).cuda()
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

        #concat the hidden state with numerical features

        out1 = torch.cat([hidden[:features.shape[0], :], features], dim=1).float()
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(out1)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

In [ ]:
device = torch.device('cuda') # define the device

size_of_vocab = len(TEXT.vocab)
aux_dim = dataloader2_iter.__next__()[0].shape[-1]
num_output_nodes = 1

In [ ]:
pip install transformers

In [ ]:
import torch.optim as optim
from transformers import AdamW

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)
    
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator1, iterator2, optimiser, criterion):
    
    #initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    #set the model in training phase
    model.train()  
    
    for batch1 in iterator1:

        #call feature iterator
        batch2 = iterator2.__next__()
        
        #resets the gradients after every batch
        optimiser.zero_grad()   
        
        #retrieve texts
        text, text_lengths = batch1.text   
        
        #get predictions
        predictions = model(text.to(device), text_lengths.to(device), batch2[0].to(device)).squeeze()
        
        #compute the loss
        loss = criterion(predictions, batch1.label.to(device))
        
        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch1.label.to(device))   
        
        #backpropage the loss and compute the gradients
        loss.backward()       
        
        #update the weights
        optimiser.step()      
        
        #loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator1), epoch_acc / len(iterator1)

In [ ]:
y = []
ypred = []

def evaluate(model, iterator1, iterator2, criterion):    
    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()
    
    #deactivates autograd
    with torch.no_grad():
    
        for batch1 in iterator1:
        
            #retrieve texts
            text, text_lengths = batch1.text

            #call feature iterator
            batch2 = iterator2.__next__()
            
            #get predictions
            predictions = model(text.to(device), text_lengths.to(device), batch2[0].to(device)).squeeze()
            
            #compute loss and accuracy
            loss = criterion(predictions, batch1.label.to(device))
            acc = binary_accuracy(predictions, batch1.label.to(device))
            y.append(batch1.label)
            ypred.append(predictions)
            
            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator1), epoch_acc / len(iterator1)

## Hyper parameter tuning

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
size_of_vocab = len(TEXT.vocab)

dropouts = list(set([0.2, 0.25]))
usebiases = list(set([True]))
num_layerses = list(set([2, 4, 6]))
l2_lamdas = list(set([0.0, 0.01]))
learning_rates = list(set([0.001, 0.0001]))
num_units = list(set([64, 128]))
mus = list(set([0.9]))
lr_sched_gammas = list(set([0, 0.2, 0.5]))
emb_dims = list(set([100]))

#######################
### HyperParameters ###
#######################
HP_dropouts = {'Drp': dropouts}
HP_usebiases = {'Bia': usebiases}
HP_num_layerses = {'nLr': num_layerses}
HP_l2_lamdas = {'L2R': l2_lamdas}
HP_learning_rates = {'LRt': learning_rates}
HP_num_units = {'NUM': num_units}
HP_mus = {'Mu': mus}
HP_sched_gammas = {'LSG': lr_sched_gammas}
HP_emb_dims = {'EmD': emb_dims}

HParamsList = [HP_dropouts, HP_usebiases, HP_num_layerses, HP_l2_lamdas, HP_learning_rates, HP_num_units, HP_mus, HP_sched_gammas, HP_emb_dims]

##############
### Metrics ##
##############
#Binary Classification
MT_Loss = {"Loss": 'BCE'}
MT_Acc = {"Metric1": 'ACC'}

MetricsList = [MT_Loss, MT_Acc]
print("Binary Classification")

Binary Classification


In [ ]:
def RunGridSearch(run_dir, hparams, metric_dict, CurLogsDir, session_num, epochs):
    with SummaryWriter(run_dir) as SummWriter:      
        print(hparams)
        
        model = classifier(size_of_vocab, aux_dim, hparams["EmD"], hparams["NUM"], num_output_nodes, hparams["nLr"], 
                        bidirectional = True, dropout = hparams["Drp"], usebias = hparams["Bia"]).to(device)
        model.embedding.weight.data.copy_(pretrained_embeddings)

        criterion = nn.BCELoss().to(device)
        optimiser = AdamW(model.parameters(), lr = hparams["LRt"], betas = (hparams["Mu"], 0.999), weight_decay = hparams["L2R"])
        scheduler = torch.optim.lr_scheduler.StepLR(optimiser, step_size = 7, gamma = hparams["LSG"])

        start_time = time.time()
        train_losses, test_losses, metric1, TrainedModel = train_test_model(hparams, metric_dict, model, criterion, optimiser, scheduler, CurLogsDir, session_num, SummWriter = SummWriter, epochs = epochs)
        elapsed_time = time.time() - start_time
        
        #text, text_lengths = next(iter(dl.Iterator(train_data, batch_size=2, shuffle=False))).text
        #num = next(iter(dataloader2._get_iterator()))[0]
        #SummWriter.add_graph(model, text.to(device), text_lengths.to(device), num.to(device)) #I think because of how your forward is, add_graph won't work for your model
        SummWriter.add_figure("LossPlot", PlotLoss(train_losses, test_losses), global_step = session_num, close = True)
        
        ModelVec.append(model)
        DurationVec.append(elapsed_time)
        
        LossVec.append(train_losses[-1])
        TestLossVec.append(test_losses[-1])
        
        AccVec.append(metric1)
        SummWriter.add_hparams(hparam_dict = hparams, metric_dict = {f"{metric_dict['Loss']}/Train": train_losses[-1], f"{metric_dict['Loss']}/Test": test_losses[-1], f"{metric_dict['Metric1']}/Test": metric1}, hparam_domain_discrete = None, run_name = str(session_num))

def PlotLoss(train_losses, test_losses):
    fig = plt.figure()
    plt.plot(train_losses, label = "Train Loss")
    plt.plot(test_losses, label = "Test Loss")
    plt.legend()
        
    return(fig)

In [ ]:
def train_test_model(hparams, metric_dict, model, criterion, optimiser, scheduler, CurLogsDir, session_num, SummWriter, epochs):
    best_valid_loss = float('inf')
    best_valid_accuracy = 0
    
    train_losses = []
    test_losses = []
    for epoch in range(epochs):
        #train the model
        dataloader2_iter = dataloader2._get_iterator()
        dataloader2_valid_iter = dataloader2_valid._get_iterator()
        train_loss, train_acc = train(model, iterator1, dataloader2_iter, optimiser, criterion)
        train_losses.append(train_loss)
  
        #evaluate the model
        valid_loss, valid_acc = evaluate(model, iterator1_valid, dataloader2_valid_iter, criterion)
        test_losses.append(valid_loss)
  
        #save the best model
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
  
        if valid_acc > best_valid_accuracy:
            best_valid_accuracy = valid_acc
            print(f'\t Best Val. Acc: {best_valid_accuracy*100:.2f}%')
            torch.save(model.state_dict(), f"saved_weights_{session_num}.pt")
  
        scheduler.step()
    
    return(train_losses, test_losses, valid_acc, model)

In [ ]:
NUM_EPOCHS = 15
session_num = 0

LossVec = []
TestLossVec = []
R2CVec = []
KLDVec = []
AccVec = []
AUCVec = []
WF1Vec = []
ModelVec = []
DurationVec = []
hparams_list = []

logdir = f"{path_root}/logs"            #CHANGE THIS to a directory on your computer
CurTime = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
CurLogsDir = logdir + "/" + CurTime + "/"
os.makedirs(CurLogsDir, exist_ok = True)

for dropout in dropouts:
    for usebias in usebiases:
        for num_layers in num_layerses:
            for l2_lamda in l2_lamdas:
                    for learning_rate in learning_rates:
                        for units in num_units:
                            for mu in mus:
                                for lr_sched_gamma in lr_sched_gammas:
                                    for emb_dim in emb_dims:
                                        hparams = dict(zip([list(item.keys())[0] for item in HParamsList], [dropout, usebias, num_layers, l2_lamda, learning_rate, units, mu, lr_sched_gamma, emb_dim]))
                                        metrics = {list(KV.keys())[0]: KV[list(KV.keys())[0]] for KV in MetricsList}
                                        hparams_list.append(hparams)

                                        os.mkdir(CurLogsDir + "run-" + str(session_num))

                                        run_name = "run-%d" % session_num
                                        print('--- Starting trial: %s' % run_name)

                                        RunGridSearch(CurLogsDir + run_name, hparams, metrics, CurLogsDir, session_num, NUM_EPOCHS)
                                        session_num += 1
print("ALL COMBINATIONS ARE NOW TRAINED!")

--- Starting trial: run-0
{'Drp': 0.2, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0.0001, 'NUM': 64, 'Mu': 0.9, 'LSG': 0, 'EmD': 100}
	 Best Val. Acc: 58.65%
	 Best Val. Acc: 58.68%
	 Best Val. Acc: 60.20%
	 Best Val. Acc: 62.31%
	 Best Val. Acc: 63.07%
	 Best Val. Acc: 66.43%
--- Starting trial: run-1
{'Drp': 0.2, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0.0001, 'NUM': 64, 'Mu': 0.9, 'LSG': 0.5, 'EmD': 100}
	 Best Val. Acc: 58.71%
	 Best Val. Acc: 60.30%
	 Best Val. Acc: 62.77%
	 Best Val. Acc: 63.34%
	 Best Val. Acc: 64.91%
	 Best Val. Acc: 66.03%
	 Best Val. Acc: 66.12%
	 Best Val. Acc: 66.49%
	 Best Val. Acc: 67.17%
	 Best Val. Acc: 67.45%
	 Best Val. Acc: 67.59%
	 Best Val. Acc: 67.67%
--- Starting trial: run-2
{'Drp': 0.2, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0.0001, 'NUM': 64, 'Mu': 0.9, 'LSG': 0.2, 'EmD': 100}
	 Best Val. Acc: 58.74%
	 Best Val. Acc: 58.77%
	 Best Val. Acc: 58.88%
	 Best Val. Acc: 60.68%
	 Best Val. Acc: 61.93%
	 Best Val. Acc: 63.00%
	 Best Val. Acc: 64.1

In [ ]:
NUM_EPOCHS = 15
session_num = 0

LossVec = []
TestLossVec = []
R2CVec = []
KLDVec = []
AccVec = []
AUCVec = []
WF1Vec = []
ModelVec = []
DurationVec = []
hparams_list = []

logdir = f"{path_root}/logs"            #CHANGE THIS to a directory on your computer
CurTime = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
CurLogsDir = logdir + "/" + CurTime + "/"
os.makedirs(CurLogsDir, exist_ok = True)

for dropout in dropouts:
    for usebias in usebiases:
        for num_layers in num_layerses:
            for l2_lamda in l2_lamdas:
                    for learning_rate in learning_rates:
                        for units in num_units:
                            for mu in mus:
                                for lr_sched_gamma in lr_sched_gammas:
                                    for emb_dim in emb_dims:
                                        hparams = dict(zip([list(item.keys())[0] for item in HParamsList], [dropout, usebias, num_layers, l2_lamda, learning_rate, units, mu, lr_sched_gamma, emb_dim]))
                                        metrics = {list(KV.keys())[0]: KV[list(KV.keys())[0]] for KV in MetricsList}
                                        hparams_list.append(hparams)

                                        os.mkdir(CurLogsDir + "run-" + str(session_num))

                                        run_name = "run-%d" % session_num
                                        print('--- Starting trial: %s' % run_name)

                                        RunGridSearch(CurLogsDir + run_name, hparams, metrics, CurLogsDir, session_num, NUM_EPOCHS)
                                        session_num += 1
print("ALL COMBINATIONS ARE NOW TRAINED!")

--- Starting trial: run-0
{'Drp': 0.2, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0.0001, 'NUM': 64, 'Mu': 0.9, 'LSG': 0, 'EmD': 100}


100%|█████████▉| 398028/400000 [00:39<00:00, 19571.39it/s]

	 Best Val. Acc: 58.74%
	 Best Val. Acc: 61.33%
	 Best Val. Acc: 62.21%
	 Best Val. Acc: 63.57%
	 Best Val. Acc: 66.04%
--- Starting trial: run-1
{'Drp': 0.2, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0.0001, 'NUM': 64, 'Mu': 0.9, 'LSG': 0.5, 'EmD': 100}
	 Best Val. Acc: 58.71%
	 Best Val. Acc: 58.77%
	 Best Val. Acc: 59.08%
	 Best Val. Acc: 61.81%
	 Best Val. Acc: 62.35%
	 Best Val. Acc: 64.12%
	 Best Val. Acc: 66.96%
	 Best Val. Acc: 68.22%
	 Best Val. Acc: 68.39%
	 Best Val. Acc: 68.48%
	 Best Val. Acc: 68.79%
	 Best Val. Acc: 68.93%
	 Best Val. Acc: 68.98%
	 Best Val. Acc: 69.13%
	 Best Val. Acc: 69.17%
--- Starting trial: run-2
{'Drp': 0.2, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0.0001, 'NUM': 64, 'Mu': 0.9, 'LSG': 0.2, 'EmD': 100}
	 Best Val. Acc: 58.71%
	 Best Val. Acc: 61.21%
	 Best Val. Acc: 62.86%
	 Best Val. Acc: 65.08%
	 Best Val. Acc: 67.72%
	 Best Val. Acc: 67.87%
	 Best Val. Acc: 67.90%
--- Starting trial: run-3
{'Drp': 0.2, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0

KeyboardInterrupt: ignored

In [ ]:
pretrained_embeddings = TEXT.vocab.vectors
size_of_vocab = len(TEXT.vocab)

dropouts = list(set([0.25]))
usebiases = list(set([True]))
num_layerses = list(set([2]))
l2_lamdas = list(set([0.0]))
learning_rates = list(set([0.0001]))
num_units = list(set([64]))
mus = list(set([0.9]))
lr_sched_gammas = list(set([0.5]))
emb_dims = list(set([100]))

#######################
### HyperParameters ###
#######################
HP_dropouts = {'Drp': dropouts}
HP_usebiases = {'Bia': usebiases}
HP_num_layerses = {'nLr': num_layerses}
HP_l2_lamdas = {'L2R': l2_lamdas}
HP_learning_rates = {'LRt': learning_rates}
HP_num_units = {'NUM': num_units}
HP_mus = {'Mu': mus}
HP_sched_gammas = {'LSG': lr_sched_gammas}
HP_emb_dims = {'EmD': emb_dims}

HParamsList = [HP_dropouts, HP_usebiases, HP_num_layerses, HP_l2_lamdas, HP_learning_rates, HP_num_units, HP_mus, HP_sched_gammas, HP_emb_dims]

##############
### Metrics ##
##############
#Binary Classification
MT_Loss = {"Loss": 'BCE'}
MT_Acc = {"Metric1": 'ACC'}

MetricsList = [MT_Loss, MT_Acc]
print("Binary Classification")

Binary Classification


In [ ]:
NUM_EPOCHS = 15
session_num = 0

LossVec = []
TestLossVec = []
R2CVec = []
KLDVec = []
AccVec = []
AUCVec = []
WF1Vec = []
ModelVec = []
DurationVec = []
hparams_list = []

logdir = f"{path_root}/logs"            #CHANGE THIS to a directory on your computer
CurTime = time.strftime("%Y-%m-%d_%H-%M-%S", time.localtime())
CurLogsDir = logdir + "/" + CurTime + "/"
os.makedirs(CurLogsDir, exist_ok = True)

for dropout in dropouts:
    for usebias in usebiases:
        for num_layers in num_layerses:
            for l2_lamda in l2_lamdas:
                    for learning_rate in learning_rates:
                        for units in num_units:
                            for mu in mus:
                                for lr_sched_gamma in lr_sched_gammas:
                                    for emb_dim in emb_dims:
                                        hparams = dict(zip([list(item.keys())[0] for item in HParamsList], [dropout, usebias, num_layers, l2_lamda, learning_rate, units, mu, lr_sched_gamma, emb_dim]))
                                        metrics = {list(KV.keys())[0]: KV[list(KV.keys())[0]] for KV in MetricsList}
                                        hparams_list.append(hparams)

                                        os.mkdir(CurLogsDir + "run-" + str(session_num))

                                        run_name = "run-%d" % session_num
                                        print('--- Starting trial: %s' % run_name)

                                        RunGridSearch(CurLogsDir + run_name, hparams, metrics, CurLogsDir, session_num, NUM_EPOCHS)
                                        session_num += 1
print("ALL COMBINATIONS ARE NOW TRAINED!")

--- Starting trial: run-0
{'Drp': 0.25, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0.0001, 'NUM': 64, 'Mu': 0.9, 'LSG': 0.5, 'EmD': 100}
	 Best Val. Acc: 58.71%
	 Best Val. Acc: 59.48%
	 Best Val. Acc: 62.36%
	 Best Val. Acc: 64.07%
	 Best Val. Acc: 67.52%
	 Best Val. Acc: 68.14%
	 Best Val. Acc: 68.73%
	 Best Val. Acc: 69.07%
	 Best Val. Acc: 69.33%
ALL COMBINATIONS ARE NOW TRAINED!


In [ ]:
session_num = 0

for dropout in dropouts:
    for usebias in usebiases:
        for num_layers in num_layerses:
            for l2_lamda in l2_lamdas:
                    for learning_rate in learning_rates:
                        for units in num_units:
                            for mu in mus:
                                for lr_sched_gamma in lr_sched_gammas:
                                    for emb_dim in emb_dims:
                                        hparams = dict(zip([list(item.keys())[0] for item in HParamsList], [dropout, usebias, num_layers, l2_lamda, learning_rate, units, mu, lr_sched_gamma, emb_dim]))
                                        print(f"{session_num}: {hparams}")
                                        session_num += 1

0: {'Drp': 0.25, 'Bia': True, 'nLr': 2, 'L2R': 0.0, 'LRt': 0.0001, 'NUM': 64, 'Mu': 0.9, 'LSG': 0.5, 'EmD': 100}


http://localhost:6006

In [ ]:
#Load the model with best valid accuracy from run number: session_num

session_num = 0
model = classifier(size_of_vocab, aux_dim, hparams_list[session_num]["EmD"], hparams_list[session_num]["NUM"], num_output_nodes, hparams_list[session_num]["nLr"], 
                        bidirectional = True, dropout = hparams_list[session_num]["Drp"], usebias = hparams_list[session_num]["Bia"]).to(device)
model.load_state_dict(torch.load(f"saved_weights_{session_num}.pt"))
model.eval()

classifier(
  (embedding): Embedding(19843, 100)
  (lstm): LSTM(100, 64, num_layers=2, batch_first=True, dropout=0.25, bidirectional=True)
  (fc): Linear(in_features=198, out_features=1, bias=True)
  (act): Sigmoid()
)

# Evaluation using the best model

In [ ]:
# Evaluate, get the predictions and accuracy
criterion = nn.BCELoss().to(device)

model = ModelVec[np.argmax(TestLossVec)]
dataloader2_test = DataLoader(TensorDataset(torch.from_numpy(num_test.values)), batch_size=64, shuffle=False)
iterator2_test = iter(dataloader2_test)


predictions = []
labels = []
model.eval()
with torch.no_grad():
    iterator1_test = dl.Iterator(test_data, batch_size=64, shuffle=False)
    for batch1 in iterator1_test:
        text, text_lengths = batch1.text
        batch2 = next(iterator2_test)
        
        predictions.append(model(text.to(device), text_lengths.to(device), batch2[0].to(device)).squeeze())
        labels.append(batch1.label.cpu().numpy())
        
iterator1_test = dl.Iterator(test_data, batch_size=64, shuffle=False)
dataloader2_test = DataLoader(TensorDataset(torch.from_numpy(num_test.values)), batch_size=64, shuffle=False)
dataloader2_test_iter = dataloader2_test._get_iterator()

_, accuracy = evaluate(model, iterator1_test, dataloader2_test_iter, criterion)

predictions = torch.cat(predictions, dim = 0).cpu().numpy()
labels = np.concatenate(labels)

In [ ]:
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 69.99%


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels, predictions >= 0.5))

              precision    recall  f1-score   support

         0.0       0.72      0.80      0.76      2549
         1.0       0.66      0.55      0.60      1792

    accuracy                           0.70      4341
   macro avg       0.69      0.68      0.68      4341
weighted avg       0.70      0.70      0.69      4341

